# Calcul the average sunrise and sunset per month
### Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime
import datetime as dt
import math
from tqdm import tqdm

### Creation of class Sun
#### Calculate the sunrise and sunset for latitude longitude and date

In [2]:
class Sun:
    def __init__(self,date):
        self.date = date
    def getSunriseTime( self, coords ):
        return self.calcSunTime( coords, True )

    def getSunsetTime( self, coords ):
        return self.calcSunTime( coords, False )

    def getCurrentUTC( self ):
        date = self.date
        return [ date.day, date.month, date.year ]

    def calcSunTime( self, coords, isRiseTime, zenith = 90.8 ):

        # isRiseTime == False, returns sunsetTime

        day, month, year = self.getCurrentUTC()

        longitude = coords['longitude']
        latitude = coords['latitude']

        TO_RAD = math.pi/180

        #1. first calculate the day of the year
        N1 = math.floor(275 * month / 9)
        N2 = math.floor((month + 9) / 12)
        N3 = (1 + math.floor((year - 4 * math.floor(year / 4) + 2) / 3))
        N = N1 - (N2 * N3) + day - 30

        #2. convert the longitude to hour value and calculate an approximate time
        lngHour = longitude / 15

        if isRiseTime:
            t = N + ((6 - lngHour) / 24)
        else: #sunset
            t = N + ((18 - lngHour) / 24)

        #3. calculate the Sun's mean anomaly
        M = (0.9856 * t) - 3.289

        #4. calculate the Sun's true longitude
        L = M + (1.916 * math.sin(TO_RAD*M)) + (0.020 * math.sin(TO_RAD * 2 * M)) + 282.634
        L = self.forceRange( L, 360 ) #NOTE: L adjusted into the range [0,360)

        #5a. calculate the Sun's right ascension

        RA = (1/TO_RAD) * math.atan(0.91764 * math.tan(TO_RAD*L))
        RA = self.forceRange( RA, 360 ) #NOTE: RA adjusted into the range [0,360)

        #5b. right ascension value needs to be in the same quadrant as L
        Lquadrant  = (math.floor( L/90)) * 90
        RAquadrant = (math.floor(RA/90)) * 90
        RA = RA + (Lquadrant - RAquadrant)

        #5c. right ascension value needs to be converted into hours
        RA = RA / 15

        #6. calculate the Sun's declination
        sinDec = 0.39782 * math.sin(TO_RAD*L)
        cosDec = math.cos(math.asin(sinDec))

        #7a. calculate the Sun's local hour angle
        cosH = (math.cos(TO_RAD*zenith) - (sinDec * math.sin(TO_RAD*latitude))) / (cosDec * math.cos(TO_RAD*latitude))

        if cosH > 1:
            return {'status': False, 'msg': 'the sun never rises on this location (on the specified date)'}

        if cosH < -1:
            return {'status': False, 'msg': 'the sun never sets on this location (on the specified date)'}

        #7b. finish calculating H and convert into hours

        if isRiseTime:
            H = 360 - (1/TO_RAD) * math.acos(cosH)
        else: #setting
            H = (1/TO_RAD) * math.acos(cosH)

        H = H / 15

        #8. calculate local mean time of rising/setting
        T = H + RA - (0.06571 * t) - 6.622

        #9. adjust back to UTC
        UT = T - lngHour
        UT = self.forceRange( UT, 24) # UTC time in decimal format (e.g. 23.23)

        #10. Return
        hr = self.forceRange(int(UT), 24)
        min = round((UT - int(UT))*60,0)

        return {
            'status': True,
            'decimal': UT,
            'hr': hr,
            'min': min 
        }

    def forceRange( self, v, max ):
        # force v to be >= 0 and < max
        if v < 0:
            return v + max
        elif v >= max:
            return v - max

        return v

### Function to get the average over a month of Sunrise and sunset

In [3]:
def get_average_sun(month,lat,long):
    date = dt.datetime(2019,month,1)
    sunrise = []
    sunset = []
    coord = {'longitude':long,'latitude':lat}
    try:
        while date.month == month:
            s = Sun(date)
            sunrise.append(s.getSunriseTime(coord)['decimal'])
            sunset.append(s.getSunsetTime(coord)['decimal'])
            date += dt.timedelta(days = 1)
        rise = np.mean(sunrise)
        set_ = np.mean(sunset)
        return rise,set_,abs(set_-rise)
    except:
        return -1,-1,0

### Apply to data (Merge database of Weather and NOAA)

In [4]:
data = pd.read_csv('/tf/Team 6 - Project/Finding_the_Biomes/Biomes/Merge_Station.csv')
data.head()


,STATION,COUNTRY,NAME,ELEVATION,LATITUDE,LONGITUDE
0,WBAF0000001,Afghanistan,Alaqahdari Dishu,595.8840,30.433333,63.300000
1,WBAF0000002,Afghanistan,Aibak,968.0448,36.266667,68.016667
2,WBAF0000003,Afghanistan,Anar Darah,782.1168,32.766667,61.650000
3,WBAF0000004,Afghanistan,Andkhoy,306.9336,36.950000,65.116667
4,WBAF0000005,Afghanistan,Arghestan,1260.9576,31.566667,66.483333


In [5]:
tqdm.pandas()
for i in range(1,13):
    data[[str(i)+'_SUNRISE',str(i)+'_SUNSET',str(i)+'_DAYLIGHT']]=data.progress_apply(lambda x: get_average_sun(i,x['LATITUDE'],x['LONGITUDE'])
                                                                             ,axis = 1,result_type = 'expand')
    
data.to_csv('/tf/Team 6 - Project/Data/Merge_Station_SSD.csv',index=False)

100%|██████████| 145393/145393 [01:17<00:00, 1878.36it/s]


In [8]:
for i in range(1,13):
    data[str(i)+'_DAYLIGHT'] = data[str(i)+'_DAYLIGHT'].progress_apply(lambda x: abs(x))

100%|██████████| 145393/145393 [00:00<00:00, 1085319.26it/s]


In [9]:
data.to_csv('/tf/Team 6 - Project/Data/Merge_Station_SSD.csv',index=False)

### Result

In [39]:
data = pd.read_csv('/tf/Team 6 - Project/Data/Merge_Station_SSD.csv')
data.head()

,STATION,COUNTRY,NAME,ELEVATION,LATITUDE,LONGITUDE,1_SUNRISE,1_SUNSET,1_DAYLIGHT,2_SUNRISE,...,9_DAYLIGHT,10_SUNRISE,10_SUNSET,10_DAYLIGHT,11_SUNRISE,11_SUNSET,11_DAYLIGHT,12_SUNRISE,12_SUNSET,12_DAYLIGHT
0,WBAF0000001,Afghanistan,Alaqahdari Dishu,595.8840,30.433333,63.300000,2.724103,13.149343,10.425240,2.463488,...,12.348806,1.827945,13.260295,11.432349,2.213604,12.846715,10.633111,2.597614,12.814545,10.216931
1,WBAF0000002,Afghanistan,Aibak,968.0448,36.266667,68.016667,2.623015,12.622509,9.999494,2.274297,...,12.414723,1.594415,12.863180,11.268766,2.083272,12.346651,10.263379,2.523761,12.259021,9.735260
2,WBAF0000003,Afghanistan,Anar Darah,782.1168,32.766667,61.650000,2.915755,13.178166,10.262411,2.621441,...,12.373613,1.969211,13.338291,11.369080,2.394330,12.885470,10.491139,2.799728,12.832395,10.032667
3,WBAF0000004,Afghanistan,Andkhoy,306.9336,36.950000,65.116667,2.843484,12.788941,9.945457,2.483317,...,12.422805,1.798105,13.045868,11.247763,2.300170,12.516281,10.216112,2.747916,12.421623,9.673707
4,WBAF0000005,Afghanistan,Arghestan,1260.9576,31.566667,66.483333,2.551029,12.898075,10.347045,2.274427,...,12.361024,1.630590,13.032926,11.402336,2.035115,12.600448,10.565333,2.429392,12.558134,10.128743


### Get Sunrise Sunset on the grid

In [4]:
data = pd.read_csv('/tf/Team 6 - Project/Data/Grid/Biomes_Elevation_Grid.csv')
data.head()


,Longitude,Latitude,Elevation,BIOME_NAME,BIOME_NUM,BIOME_CODE,Coord
0,6.3,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,POINT (6.299999999989411 -86.90000000000019)
1,6.5,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,POINT (6.499999999989399 -86.90000000000019)
2,6.6,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,POINT (6.599999999989393 -86.90000000000019)
3,6.7,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,POINT (6.699999999989388 -86.90000000000019)
4,6.8,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,POINT (6.799999999989383 -86.90000000000019)


In [5]:
data.drop(columns='Coord',inplace = True)

In [ ]:
tqdm.pandas()
for i in range(2,13):
    data[[str(i)+'_SUNRISE',str(i)+'_SUNSET',str(i)+'_DAYLIGHT']]=data.progress_apply(lambda x: get_average_sun(i,x['Latitude'],x['Longitude'])
                                                                             ,axis = 1,result_type = 'expand')
    data.to_csv('/tf/Team 6 - Project/Data/Grid/full_grid.csv',index = False)
    


 86%|████████▌ | 1323730/1543684 [51:45<33:42, 108.75it/s]    

In [8]:
data.head()

,Longitude,Latitude,Elevation,BIOME_NAME,BIOME_NUM,BIOME_CODE,1_SUNRISE,1_SUNSET,1_DAYLIGHT
0,6.3,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,-1.0,-1.0,0.0
1,6.5,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,-1.0,-1.0,0.0
2,6.6,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,-1.0,-1.0,0.0
3,6.7,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,-1.0,-1.0,0.0
4,6.8,-86.9,0.0,"Mediterranean Forests, Woodlands & Scrub",12.0,AU12,-1.0,-1.0,0.0
